In [1]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain_ollama import OllamaLLM
from langchain import PromptTemplate
from langchain_community.document_loaders import TextLoader
from langchain.schema import Document
from langgraph.prebuilt import create_react_agent
import datetime
import numpy as np
import time
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder


In [2]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [3]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH00-RAG-Chatbot")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH00-RAG-Chatbot


In [4]:
OLLAMA_BASE_URL = "http://localhost:11434" # Ollama 서버 주소
OLLAMA_CHAT_MODEL = "ollama-ko-0710:latest" # Ollama에서 실행중인 모델 이름

In [10]:
# 문서 출력 도우미 함수
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )
    

## 기본 버전

In [4]:
def general_llm():
    # 3. 텍스트 청크 분할 (chunk_size=1000, chunk_overlap=50)
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
    loader = TextLoader("./data/text.txt", encoding="utf-8")
    split_docs = loader.load_and_split(text_splitter)
    model_name = "intfloat/multilingual-e5-large-instruct"
    hf_embeddings = HuggingFaceEmbeddings(
        model_name=model_name,
        model_kwargs={"device": "cuda"},  # cuda, cpu
        encode_kwargs={"normalize_embeddings": True},
    )

    db = FAISS.from_documents(documents=split_docs, embedding=hf_embeddings)

    # 5. 리트리버 생성
    retriever = db.as_retriever(
        search_type="mmr", search_kwargs={"k": 10, "lambda_mult": 0.25, "fetch_k": 20}
    )

    template = """당신은 유용한 AI 어시스턴트입니다. 사용자의 질의에 대해 친절하고 정확하게 답변해야 합니다.
    You are a helpful AI assistant, you'll need to answer users' queries in a friendly and accurate manner.
    모든 대답은 반드시 한국말로 대답해주세요.
    
    문서 내용:
    {context}
    질문: {question}
    답변:"""
    prompt = PromptTemplate(template=template, input_variables=["context", "question"])
    
    # # 7. Ollama LLM 초기화
    # llm = OllamaLLM(model=OLLAMA_CHAT_MODEL, base_url=OLLAMA_BASE_URL,temperature=1, max_tokens=1024)
    llm = ChatOpenAI(model_name="gpt-4.1-mini", temperature=0.7, max_tokens=1024)

    # # 8. RAG 체인 생성
    chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        return_source_documents=True,
        chain_type_kwargs={"prompt": prompt}
    )

    result = chain.invoke({"query": "삼성전자 상반기 주가 실적?"})
    answer = result["result"]
    return answer

In [5]:
general_llm()

'삼성전자의 2024년 상반기 주가 실적은 전반적으로 상승세를 보이고 있습니다. 9월 중순 기준으로 삼성전자 주가는 8만원대에 근접하며, 16일에는 전날보다 3.66% 오른 7만9300원에 거래를 마감했습니다. 투자자 중 외국인과 기관이 대규모로 삼성전자 주식을 매수하며 주가 상승을 견인했습니다. 특히 외국인은 하루 동안 1조 원 이상을 매수했고, 기관도 약 692억 원어치를 순매수했습니다. \n\n영업 실적 측면에서는 삼성전자가 올해 상반기 국내 1위 영업이익을 기록한 SK하이닉스와 함께 반도체 업종의 강세가 두드러졌습니다. 미래에셋증권은 삼성전자의 올해와 내년 영업이익 전망치를 각각 상향 조정했으며, 주가 목표도 상승시켰습니다. 반도체 공급 부족과 고대역폭 메모리(HBM) 생산 확대가 메모리 반도체 가격 상승을 이끌면서 실적 개선에 긍정적인 영향을 미치고 있습니다.\n\n요약하자면, 삼성전자 주가는 외국인과 기관의 강한 매수세에 힘입어 상반기 이후 꾸준히 상승 중이며, 반도체 실적 개선과 공급 제한에 따른 가격 상승 기대감이 주가 상승을 뒷받침하고 있습니다.'